# नोटबुक 2: PP-OCR के साथ टेक्स्ट डिटेक्शन मॉडल को ट्रेनिंग देना 🚀

**उद्देश्य:** इस नोटबुक का उद्देश्य `1_Data_Preparation.ipynb` में तैयार किए गए डेटा का उपयोग करके एक टेक्स्ट डिटेक्शन मॉडल को प्रशिक्षित करना है। हम कॉन्फ़िगरेशन फ़ाइल को संशोधित करेंगे, ट्रेनिंग शुरू करेंगे, और प्रशिक्षित मॉडल के साथ अनुमान (inference) चलाएंगे।

---

## चरण 1: परिवेश सेटअप
यह सुनिश्चित करें कि आपके पास एक GPU सक्षम वातावरण है। हम PaddlePaddle और PaddleOCR स्थापित करेंगे, और ट्रेनिंग स्क्रिप्ट के लिए रिपॉजिटरी को क्लोन करेंगे।

In [ ]:
# PaddlePaddle और PaddleOCR स्थापित करें
!python -m pip install paddlepaddle-gpu -f https://www.paddlepaddle.org.cn/whl/linux/mkl/avx/stable.html
!pip install "paddleocr>=2.7" pyyaml

# ट्रेनिंग के लिए PaddleOCR रिपॉजिटरी को क्लोन करें
!git clone https://github.com/PaddlePaddle/PaddleOCR.git
%cd PaddleOCR

---

## चरण 2: डेटा को लिंक करना
हम अपने तैयार किए गए डेटासेट को PaddleOCR निर्देशिका में लिंक करेंगे ताकि स्क्रिप्ट उसे ढूँढ सके।

In [ ]:
# मान लें कि आपका 'dataset' फ़ोल्डर एक स्तर ऊपर है
!mkdir -p train_data
!ln -s ../dataset ./train_data/icdar2015 # एक सामान्य नाम के रूप में लिंक करें

---

## चरण 3: ट्रेनिंग कॉन्फ़िगरेशन को संशोधित करना
हम एक हल्के डिटेक्शन मॉडल (`det_mv3_db.yml`) के लिए कॉन्फ़िगरेशन को संशोधित करेंगे ताकि वह हमारे कस्टम डेटासेट को इंगित करे और ट्रेनिंग मापदंडों को समायोजित करे।

In [ ]:
import yaml

config_path = 'configs/det/det_mv3_db.yml'
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# हमारे डेटासेट पथ सेट करें
config['Global']['save_model_dir'] = './output/det_db'
config['Global']['epoch_num'] = 50 # प्रदर्शन के लिए युगों की संख्या कम करें
config['Optimizer']['lr']['learning_rate'] = 0.0005

config['Train']['dataset']['data_dir'] = './train_data/icdar2015/'
config['Train']['dataset']['label_file_list'] = ['./train_data/icdar2015/det_label_train.txt']

config['Eval']['dataset']['data_dir'] = './train_data/icdar2015/'
config['Eval']['dataset']['label_file_list'] = ['./train_data/icdar2015/det_label_val.txt']

# संशोधित कॉन्फ़िगरेशन को एक नई फ़ाइल में सहेजें
custom_config_path = 'configs/det/my_custom_det_config.yml'
with open(custom_config_path, 'w') as f:
    yaml.dump(config, f)

print(f"कस्टम कॉन्फ़िगरेशन '{custom_config_path}' में सहेजा गया।")

---

## चरण 4: ट्रेनिंग शुरू करना
अब हम अपनी कस्टम कॉन्फ़िगरेशन फ़ाइल का उपयोग करके `tools/train.py` स्क्रिप्ट चलाएंगे। हम एक पूर्व-प्रशिक्षित बैकबोन का उपयोग करके ट्रेनिंग को गति देंगे।

In [ ]:
# पूर्व-प्रशिक्षित मॉडल डाउनलोड करें (यदि आवश्यक हो)
!wget https://paddle-ocr-pretrain.bj.bcebos.com/MobileNetV3_large_x0_5_pretrained.pdparams -P ./pretrain_models/

# ट्रेनिंग कमांड चलाएँ
!python tools/train.py \
    -c configs/det/my_custom_det_config.yml \
    -o Global.pretrained_model=./pretrain_models/MobileNetV3_large_x0_5_pretrained

---

## चरण 5: प्रशिक्षित मॉडल के साथ अनुमान (Inference)
ट्रेनिंग पूरी होने के बाद, हम अपने सबसे अच्छे मॉडल को लोड करेंगे और एक परीक्षण छवि पर इसका उपयोग करेंगे।

In [ ]:
from paddleocr import PaddleOCR, draw_ocr
from PIL import Image

# PaddleOCR को प्रशिक्षित डिटेक्शन मॉडल के साथ प्रारंभ करें
# केवल डिटेक्शन के लिए rec=False सेट करें
ocr_engine = PaddleOCR(
    det_model_dir='./output/det_db/best_accuracy',
    use_angle_cls=False,
    rec=False, # हम केवल डिटेक्शन का परीक्षण कर रहे हैं
    use_gpu=True
)

img_path = './train_data/icdar2015/images/img_01.png'
result = ocr_engine.ocr(img_path, cls=False, det=True)

# परिणाम की कल्पना करें
print("डिटेक्शन परिणाम:")
print(result)

image = Image.open(img_path).convert('RGB')
boxes = [line[0] for line in result[0]]
im_show = draw_ocr(image, boxes)
display(Image.fromarray(im_show))